# Análisis del riesgo de incumplimiento de los prestatarios

## Índice <a id='back'></a>
* [Introducción](#intro)
* [Etapa 1. Descripción de los datos](#data_review)
    * [1.1. Descripción de los datos](#data_review_data_review)
    * [1.2. Valores ausentes](#data_review_missing_values)
    * [1.3. Conclusiones](#data_review_conclusions)
* [Etapa 2. Preprocesamiento de datos](#data_preprocessing)
    * [2.1. Duplicados implícitos y datos problemáticos](#data_preprocessing_duplicated)
    * [2.2. Duplicados obvios](#data_preprocessing_duplicated_ob)
    * [2.3. Completar valores ausentes](#data_preprocessing_missing_values)
        * [2.3.1 Restaurar valores ausentes en `'total_income'`](#data_preprocessing_missing_values_income)
        * [2.3.2 Restaurar valores ausentes en `'days_employed'`](#data_preprocessing_missing_values_days)
    * [2.4. Conclusiones](#data_preprocessing_conclusions)
* [Etapa 3. Comprobación de hipótesis](#hypotheses)
    * [3.1. Hipótesis: cantidad de hijos](#hypotheses_children)
    * [3.2. Hipótesis: estado civil](#hypotheses_family_status)
    * [3.3. Hipótesis: ingreso mensual](#hypotheses_family_status)
    * [3.4. Hipótesis: propósito del prestámo](#hypotheses_purpose)
    * [3.5. Conclusiones](#hypotheses_conclusions)
* [Conclusión general](#conclusions)

## Introducción <a id='intro'></a>
Este proyecto consiste en preparar un informe para la división de préstamos de un banco. Se averigua si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

### Objetivo e hipótesis:
Este informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. Dicha puntuación se utilizará para evaluar la capacidad de un prestatario potencial para pagar su préstamo.
Hipótesis: 
1. El incumplimiento de pago de un préstamo de los clientes difiere según el estado civil, cantidad de hijos, ingreso mensual o propósito del pago. 
2. Los clientes con una cantidad menor de hijos forman el mayor porcentaje de pago a tiempo.
3. Los clientes casados forman el mayor porcentaje de pago a tiempo.
4. Los clientes con mayores ingresos forman el mayor porcentaje de pago a tiempo.
5. Los propósitos del préstamo no interfieren con el pago a tiempo.

### Etapas 
Los datos del cliente se almacenan en el archivo `/datasets/credit_scoring_eng.csv`. No hay ninguna información sobre la calidad de los datos así que se examinarán antes de probar las hipótesis.

Primero, se evaluará la calidad de los datos y si los problemas son significativos. Entonces, durante el preprocesamiento de datos, se tomará en cuenta los problemas más críticos y finalmente se comprobarán las hipótesis para presentar las conclusiones.
 
El proyecto consistirá en cuatro etapas:
 1. Descripción de los datos.
 2. Preprocesamiento de datos.
 3. Comprobación de hipótesis.
 4. Conclusiones.

[Volver a Contenidos](#back)

## Etapa 1. Descripción de los datos <a id='data_review'></a>

Se importan las librerías necesarias.

In [1]:
import pandas as pd

Se lee el archivo `credit_scoring_eng.csv` de la carpeta `/datasets/` y se guarda en la variable `df`.

In [2]:
try:
    df = pd.read_csv('credit_scoring_eng.csv')
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

Se obtienen las columnas y se imprimen las primeras filas para obtener información de su contenido.

In [3]:
print(df.columns)
print()
print(df.head())

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

   children  days_employed  dob_years            education  education_id  \
0         1   -8437.673028         42    bachelor's degree             0   
1         1   -4024.803754         36  secondary education             1   
2         0   -5623.422610         33  Secondary Education             1   
3         3   -4124.747207         32  secondary education             1   
4         0  340266.072047         53  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1            married                 0      F    employee     0     17932.802   
2            married                 0      M    employee     0     23341.752   
3     

### 1. 1. Descripción de los datos <a id='data_review_data_review'></a>

La tabla contiene doce columnas. De acuerdo con los datos de las primeras filas se tiene:

- `children` - cantidad de hijos del cliente.
- `days_employed` - experiencia laboral del cliente en días (con valores problemáticos).
- `dob_years` - edad del cliente en años.
- `education` - nivel educativo del cliente (con duplicados implícitos).
- `education_id` - identificador del nivel educativo del cliente.
- `family_status` - estado civil del cliente.
- `family_status_id` - identificador de estado civil del cliente.
- `gender` - género del cliente.
- `income_type` - tipo de empleo del cliente.
- `debt` - existencia de alguna deuda por parte del cliente en el pago de un préstamo.
- `total_income` - ingreso mensual del cliente.
- `purpose` - propósito del préstamo otorgado al cliente.

No existen problemas con el estilo de los nombres de las columnas. En las primeras 5 filas no parece haber duplicados obvios, sin embargo no se cuenta con un identificador de cliente por lo que es factible que existan duplicados, se investigará más a detalle en la etapa de preprocesamiento de datos.

Se procede a obtener información de la base de datos.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


La base de datos cuenta con 21525 filas, 12 columnas como ya se había mencionado. Dos columnas contienen datos tipo float, cinco contienen datos tipo int y las restantes contienen datos tipo object. Además, observamos que las columnas `'days_employed'` y `'total_income'` contienen menos datos que la cantidad de filas. Esto es, ambas columnas tienen valores ausentes.

### 1. 2. Valores ausentes <a id='data_review_missing_values'></a>

Se obtienen los valores ausentes de cada columnas.

In [5]:
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Como se había mencionado, sólo dos columnas contienen valores ausentes y ambas tienen la misma cantidad de ellos. Se filtrará la tabla con los valores ausentes de `'days_employed'` y se imprimirán las primeras filas para observación.

In [6]:
df_filtered_day = df.loc[df['days_employed'].isna()]
print(df_filtered_day.head())

    children  days_employed  dob_years            education  education_id  \
12         0            NaN         65  secondary education             1   
26         0            NaN         41  secondary education             1   
29         0            NaN         63  secondary education             1   
41         0            NaN         50  secondary education             1   
55         0            NaN         54  secondary education             1   

        family_status  family_status_id gender    income_type  debt  \
12  civil partnership                 1      M        retiree     0   
26            married                 0      M  civil servant     0   
29          unmarried                 4      F        retiree     0   
41            married                 0      F  civil servant     0   
55  civil partnership                 1      F        retiree     1   

    total_income                   purpose  
12           NaN         to have a wedding  
26           NaN    

Estas primeras cinco filas también contienen valores ausentes en la columna `'total_income'`. Debido a que ambas columnas contienen la misma cantidad de valores ausentes, se obtendrá la cantidad de valores ausentes de esta tabla filtrada en la columna `'total_income'`.

In [7]:
print('Valores ausentes en la columna total_income:', df_filtered_day['total_income'].isna().sum())

Valores ausentes en la columna total_income: 2174


Es la misma cantidad que en la tabla original. Esto significa que las filas con valores ausentes en la columna `'days_employed'` son las mismas que las que contienen valores ausentes en la columna `'total_income'`.

Así, en la tabla original, la cantidad de filas con valores ausentes es 2174, lo que representa el 10.09% de las 21525 filas en total. Al ser un porcentaje considerable se quiere completar los valores ausentes. Para hacer eso, se debe definir si estos valores ausentes se deben a una característica específica del cliente o si son aleatorios.

Las columnas con datos relacionados a la experiencia laboral del cliente (`'days_employed'`) son: `'income_type'` y `'total_income'`. Como `'total_income'` es la otra columna con valores ausentes, se investigarán los datos de `'income_type'`.

Una razón de valores ausentes en estas dos columnas puede ser que el cliente es desempleado y por ello no cuenta con los datos de experiencia laboral e ingreso mensual.

In [8]:
df_grouped_nan = df_filtered_day['income_type'].value_counts()
print(df_grouped_nan)

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64


Los clientes con `'unemployed'` no pertenecen a las filas con valores ausentes, así que esta característica no influye con los valores ausentes.

Se imprimirá la distribución de los valores de la columna `'income_type'` de la tabla original.

In [9]:
print(df.groupby('income_type').size().sort_values(ascending=False))

income_type
employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
dtype: int64


La ditribución de los valores en `'income_type'` en la tabla original es similar a su distribución en las filas con valores ausentes. Salvo el cliente con valores ausentes y tipo de empleo emprendedor, todos los demás representan aproximadamente el 10% de su categoría (tipo de empleo).

### 1. 3. Conclusiones <a id='data_review_conclusions'></a> 

Cada fila de la tabla almacena datos del cliente que solicitó el prestámo. La mayoría de columnas describen al cliente en sí: número de hijos, estado civil, ingreso mensual, etc. Dos columnas decriben el estado del préstamo: su propósito y si existe alguna deuda en el pago.

Está claro que los datos son suficientes para probar las hipótesis. Sin embargo, hay valores ausentes en dos columnas que no están relacionados con una característica específica del cliente. Así, los valores ausentes son aleatorios.

Para continuar, es necesario preprocesar los datos.

[Volver a Contenidos](#back)

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing'></a>

En esta etapa se investigarán duplicados y se completarán valores ausentes. Lo primero a invesigar son duplicados implícitos en las columnas para corregir, así como datos problemáticos.

Posteriormente se completarán los valores ausentes con las medias o medianas de los datos, dependiendo de las características del cliente que puedan influir para los valores correspondientes.

### 2. 1. Duplicados implícitos y datos problemáticos <a id='data_preprocessing_duplicated'></a>

Se imprimirá la distribución de los valores de la primera columna `'children'`, para determinar si hay datos problemáticos. Los valores deben ser números enteros positivos, o bien, igual a cero.

In [10]:
print(df['children'].value_counts().sort_index())

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64


Existen 47 filas con el dato problemático -1, esto puede deberse a un error humano. Se obtendrán los porcentajes de esta distribución.

In [11]:
print((df['children'].value_counts().sort_index() / len(df)) * 100)

-1      0.218351
 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 4      0.190476
 5      0.041812
 20     0.353078
Name: children, dtype: float64


El dato problemático representa el 0.2% de todos los datos. Al ser un porcentaje muy bajo, se reemplazará este valor por el valor de los datos con mayor porcentaje, en este caso, 0.

In [12]:
df['children'] = df['children'].replace(-1, 0)
print(df['children'].value_counts().sort_index())

0     14196
1      4818
2      2055
3       330
4        41
5         9
20       76
Name: children, dtype: int64


Se imprimen los valores de la columna siguiente `'days_employed'` para detectar datos problemáticos. Estos datos son de tipo float y por ser la cantidad de días de experiencia laboral deben ser números tipo float positivos, o bien, igual a cero.

In [13]:
print(df['days_employed'].value_counts().sort_index())

-18388.949901     1
-17615.563266     1
-16593.472817     1
-16264.699501     1
-16119.687737     1
                 ..
 401663.850046    1
 401674.466633    1
 401675.093434    1
 401715.811749    1
 401755.400475    1
Name: days_employed, Length: 19351, dtype: int64


Existen números negativos. Se filtrará la tabla con estos datos para obtener el porcentaje y decidir el procedimiento para tratarlos.

In [14]:
negative_days = len(df[df['days_employed'] < 0]) # cantidad de filas con días negativos
conversion_negative_days = negative_days / len(df) # razón de días negativos sobre la cantidad total
print(f'Hay {negative_days} filas con días negativos, esto es el {conversion_negative_days:.0%}')

Hay 15906 filas con días negativos, esto es el 74%


El 74% es un porcentaje muy alto de datos problemáticos en esta columna. La razón más obvia parece ser que el cliente haya realizado la diferencia entre la fecha en la que comenzó a trabajar y la fecha actual, por ello la cantidad negativa. Se reemplazará el signo negativo por positivo.

Para ello se aplicará la función `abs()` de Python en la columna `'days_employed`'.

In [15]:
df['days_employed'] = df['days_employed'].apply(abs)
print(df['days_employed'].value_counts().sort_index())

24.141633        1
24.240695        1
30.195337        1
33.520665        1
34.701045        1
                ..
401663.850046    1
401674.466633    1
401675.093434    1
401715.811749    1
401755.400475    1
Name: days_employed, Length: 19351, dtype: int64


La siguiente columna a investigar es `'dob_years'`. Un cliente debe tener la mayoría de edad para solicitar un prestámo, de esta manera los valores problemáticos serán aquellos menores de 18 años.

In [16]:
print(df['dob_years'].value_counts().sort_index())

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64


El único valor problemático es 0. Se obtendrán la media y la mediana (sin contar los datos problemáticos) para decir por cual de ellas se reemplezará.

In [17]:
df_filtered_19 = df[df['dob_years'] > 18]
mean_years = int(df_filtered_19['dob_years'].mean())
median_years = int(df_filtered_19['dob_years'].median())
print(f'La media es {mean_years} y la mediana es {median_years}')

La media es 43 y la mediana es 43


Ambos valores coinciden por lo que se reemplazará el valor 0 por 43.

In [18]:
df['dob_years'] = df['dob_years'].replace(0, mean_years)
print(df['dob_years'].value_counts().sort_index())

19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    614
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64


La columna siguiente es `'education'`. Como los valores son de tipo object, la problemática puede ser que difieran por mayúsculas o por ortografía. Por esta razón se imprimirán los valores únicos de esta columna.

In [19]:
print(df['education'].unique())

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


La diferencia de los valores es el uso de las mayúsculas. Se cambiarán los valores de esta columna por letras minúsculas.

In [20]:
df['education'] = df['education'].str.lower()
print(df['education'].value_counts())

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64


La siguiente columna `'education_id'` contienen los identificadores del nivel educativo de los clientes. Se imprimirá la distirbución de los valores para comparar con la distibución anterior y crear un diccionario por si es necesario trabajar con ello más adelante.

In [21]:
print(df['education_id'].value_counts())

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


In [22]:
education_dict = {
    'secondary education': 1,
    'bachelor\'s degree': 0,
    'some college': 2,
    'primary education': 3,
    'graduate degree': 4
}

La siguiente columna es `'family_status'` y como los valores son de tipo object, se tratará como la columna `'education'`.

In [23]:
print(df['family_status'].unique())

['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


No hay dúplicados implícitos. Se imprimirá su distribución.

In [24]:
print(df['family_status'].value_counts())

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64


Se imprimirá la distribución de la columna siguiente `'family_status_id'` para comparar con la anterior y crear el diccionario correspondiente.

In [25]:
print(df['family_status_id'].value_counts())

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


In [26]:
family_status_dict = {
    'married': 0,
    'civil partnership': 1,
    'unmarried': 4,
    'divorced': 3,
    'widow / widower': 2
}

La siguiente columna `'gender'` tamnbién contiene valores tipo object. Se imprimirá sus valores únicos.

In [27]:
print(df['gender'].unique())

['F' 'M' 'XNA']


Aparece el valor `'XNA'`, esto puede ser por clientes de género no binario. Como esta variable no afecta a las hipótesis, entonces no se tendrá como un dato problemático. No se modifica nada en esta columna.

La columna siguiente `'income_type'`contiene valores tipo object, se procederá como en las anteriores.

In [28]:
print(df['income_type'].unique())

['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']


No hay datos problemáticos con esta columna. No se modifica nada.

La siguiente columna `'debt'` contiene valores de tipo int, se imprimrá su distribución.

In [29]:
print(df['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


Sólo contiene dos tipos de valores, 0 y 1. Esto es equivalente a tener No y Sí (respectivamente) como respuesta a la pregunta ¿el cliente ha tenido alguna deuda en el pago de un prestámo?

No contiene datos problemáticos por lo que no se modifica nada en esta columna.

La siguiente columna `'total_income'` contiene valores del tipo float. Como esta columna corresponde al ingreso mensual del cliente, los valores deben ser números positivos o iguales a cero.

In [30]:
print(df['total_income'].value_counts().sort_index())

3306.762      1
3392.845      1
3418.824      1
3471.216      1
3503.298      1
             ..
273809.483    1
274402.943    1
276204.162    1
352136.354    1
362496.645    1
Name: total_income, Length: 19348, dtype: int64


No hay datos problemáticos por lo que no se modifica nada en esta columna.

La última columna `'purpose'` contiene valores de tipo object. Se imprimirán los valores únicos.

In [31]:
print(df['purpose'].unique())

['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property for renting out' 'to buy a car' 'housing renovation'
 'going to university']


Existen duplicados implícitos, muchos de ellos corresponden al mismo propósito pero usando sinónimos o difiriendo por alguna palabra. Se crearán las siguientes listas:
* `real_state` - contiene los propósitos relacionados con la compra, renta, reparación o renovación de una propiedad.
* `buy_car` - contiene los propósitos relacionados con la compra de un auto.
* `education` - contiene los propósitos relacionados con educación.
* `wedding` - contiene los propósitos relacionados con gastos de una boda.

In [32]:
real_state = [
    'purchase of the house',
    'housing transactions',
    'purchase of the house for my family',
    'buy real estate',
    'buy commercial real estate',
    'buy residential real estate',
    'construction of own property',
    'property',
    'building a property',
    'transactions with commercial real estate',
    'building a real estate',
    'housing',
    'transactions with my real estate',
    'purchase of my own house',
    'real estate transactions',
    'buying property for renting out',
    'housing renovation'
]
buy_car = [
    'car purchase',
    'buying a second-hand car',
    'buying my own car',
    'cars',
    'second-hand car purchase',
    'car',
    'to own a car',
    'purchase of a car',
    'to buy a car'
]
education = [
    'supplementary education',
    'education',
    'to become educated',
    'getting an education',
    'to get a supplementary education',
    'getting higher education',
    'profile education',
    'university education',
    'going to university'
]
wedding = [
    'to have a wedding',
    'having a wedding',
    'wedding ceremony'
]

Se creará una función que envía cada valor a su lista representativa.

In [33]:
def assign_purpose(string):
    if string in real_state:
        return 'for real state'
    elif string in buy_car:
        return 'to buy a car'
    elif string in education:
        return 'for education'
    elif string in wedding:
        return 'for a wedding'
    else:
        return string

Se aplicará la función a la columna `'purpose'` de la tabla para reemplazar los datos.

In [34]:
df['purpose'] = df['purpose'].apply(assign_purpose)

Se comprueba que en la columna `'purpose'` se tengan 4 valores únicos.

In [35]:
print(df['purpose'].unique())

['for real state' 'to buy a car' 'for education' 'for a wedding']


[Volver a Contenidos](#back)

### 2. 2. Duplicados obvios <a id='data_preprocessing_duplicated_ob'></a>

Se obtendrá la cantidad de filas duplicadas y su porcentaje.

In [36]:
duplicated_rows = df.duplicated().sum()
conv_duplicated_rows = duplicated_rows / len(df) # razón entre la cantidad de filas duplicadas y la cantidad total de filas
print(f'Existen {duplicated_rows} filas duplicadas, esto es el {conv_duplicated_rows:.0%}')

Existen 409 filas duplicadas, esto es el 2%


El 2% de las filas de la tabla se ha duplicado. Como los valores de `'days_employed'` son de tipo float, se puede usar este valor para distinguir un poco más si se trata del mismo cliente.

Se filtrará la tabla con las filas duplicadas y se imprimirán los valores de ésta contenidos en la columna `'days_employed'` (incluyendo valores ausentes).

In [37]:
df_duplicates = df[df.duplicated(keep='first')]
print(df_duplicates['days_employed'].value_counts(dropna=False))

NaN    409
Name: days_employed, dtype: int64


Todas las filas tienen valor ausente en la columna `'days_employed'` (y por tanto en la columna `'total_income'`). Como no se tiene un identificador de cliente, se desconoce si las filas duplicadas pertenecen a un mismo cliente o no.

Se decide conservar estas filas y tratarlas como clientes distintos ya que el porcentaje supera el 1%.

[Volver a Contenidos](#back)

### 2. 3. Completar valores ausentes <a id='data_preprocessing_missing_values'></a>

Como se ha mencionado, los valores ausentes que se tienen de las columnas `'days_employed'` y `'total_income'` pertenecen a las mismas filas y además son aleatorios.

En esta sección se restaurarán esos valores ausentes con la mediana o la media de grupos correspondientes. Para ello se considerarán las características que influyan con los datos de las columnas mencionadas.

Se agruparán los valores de `'dob_years'` en grupos de edad por si es necesario considerar la edad como un factor que influya en los valores de las columnas a tratar. Para ello se creará un función que envié cierta edad a su grupo correspondiente.

In [38]:
def age_group(age):
    if age < 20:
        return '< 20'
    elif 20 <= age < 30:
        return '20-29'
    elif 30 <= age < 40:
        return '30-39'
    elif 40 <= age < 50:
        return '40-49'
    elif 50 <= age < 60:
        return '50-59'
    elif 60 <= age < 70:
        return '60-69'
    else:
        return '70+'

Se comprueba que la función sea correcta.

In [39]:
print(age_group(19))
print(age_group(29))
print(age_group(39))
print(age_group(49))
print(age_group(59))
print(age_group(69))
print(age_group(89))

< 20
20-29
30-39
40-49
50-59
60-69
70+


Se crea una nueva columna `'age_group'` para la tabla original que contiene el grupo de edad para cada cliente. Para ello se aplica la función recién creada a la columna `'dob_years'`. Se obtiene la información de la tabla para corroborar que los cambios se hicieron de manera correcta.

In [40]:
df['age_group'] = df['dob_years'].apply(age_group)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
 12  age_group         21525 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


La conjetura es que la edad del cliente y la experiencia laboral son proporcionales, esto es, a mayor edad, mayor experiencia laboral.
Para comprobar la veracidad de esta conjetura, se agrupará la tabla por categoría de edad y se obtendrá la media de la cantidad de días de experiencia laboral.

In [41]:
df_grouped_age = df.groupby('age_group')['days_employed'].mean()
print(df_grouped_age)

age_group
20-29      2089.054192
30-39      4155.029251
40-49     13439.227108
50-59    132907.545543
60-69    283926.481689
70+      320819.151927
< 20        633.678086
Name: days_employed, dtype: float64


La conjetura es cierta:

**a mayor edad, mayor es la cantidad de días de experiencia laboral.**

Ahora se creará una tabla a partir de la original pero sin valores ausentes para tener mayor certeza al calcular medias y medianas. Se obtiene la información de la nueva tabla para corroborar que no contiene valores ausentes.

In [42]:
df_non_nan = df.dropna()
df_non_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          19351 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         19351 non-null  int64  
 3   education         19351 non-null  object 
 4   education_id      19351 non-null  int64  
 5   family_status     19351 non-null  object 
 6   family_status_id  19351 non-null  int64  
 7   gender            19351 non-null  object 
 8   income_type       19351 non-null  object 
 9   debt              19351 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           19351 non-null  object 
 12  age_group         19351 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


#### 2.3.1 Restaurar valores ausentes en `'total_income'` <a id='data_preprocessing_missing_values_income'></a>

En primer lugar se restaurarán los valores ausentes de la columna `'total_income'`.

El ingreso mensual de un cliente depende de tres facores: el nivel educativo, la experiencia laboral y el puesto. Como la experiencia laboral es proporcional a la edad del cliente, se considerarán tablas dinámicas considerando estos tres factores y obteniendo las medias y medianas de los ingresos mensuales.

In [43]:
#Tabla dinámica con las medias
data_pivot_mean = df_non_nan.pivot_table(index=['age_group', 'education'], columns='income_type', values='total_income', aggfunc='mean')
print(data_pivot_mean)

income_type                        business  civil servant      employee  \
age_group education                                                        
20-29     bachelor's degree    32256.239769   27907.014075  27706.839102   
          primary education    30460.981833   30563.383000  26614.028556   
          secondary education  26116.903475   22601.223783  22612.924227   
          some college         27855.456737   21662.198364  23860.788568   
30-39     bachelor's degree    39169.886242   32040.193301  30861.492968   
          graduate degree               NaN   17822.757000  18551.846000   
          primary education    25068.715417   21150.696000  21459.375547   
          secondary education  29227.042915   24654.951415  24560.505804   
          some college         32208.165662   32407.862250  31785.952667   
40-49     bachelor's degree    41067.908317   33339.033856  31530.697077   
          graduate degree               NaN            NaN  31771.321000   
          pr

In [44]:
# Tabla dinámica con las medianas
data_pivot_median = df_non_nan.pivot_table(index=['age_group', 'education'], columns='income_type', values='total_income', aggfunc='median')
print(data_pivot_median)

income_type                      business  civil servant    employee  \
age_group education                                                    
20-29     bachelor's degree    28250.2840     24617.5440  24030.5850   
          primary education    24210.1960     30563.3830  24583.5965   
          secondary education  23884.9040     21361.7300  20192.6190   
          some college         25429.2090     21640.4795  21907.8130   
30-39     bachelor's degree    32574.9760     27956.2445  26722.5325   
          graduate degree             NaN     17822.7570  18551.8460   
          primary education    20030.2260     21150.6960  19546.3410   
          secondary education  26268.0480     20870.9730  22088.2215   
          some college         29814.5275     30672.7180  26913.8230   
40-49     bachelor's degree    34604.4780     28425.4810  27266.6900   
          graduate degree             NaN            NaN  31771.3210   
          primary education    27929.0980     78410.7740  21132.

Se observa que las medias difieren de las medianas, esto significa que existen valores atípicos, por lo que se usarán las medianas.

Se creará una función que analicé cada fila de la tabla original y asigne la mediana correspondiente.

In [45]:
def nan_median(row):
    income = row['total_income']
    age = row['age_group']
    education = row['education']
    in_type = row['income_type']
    if pd.isna(income):
        income = data_pivot_median.loc[(age, education), in_type]
        return income
    else:
        return income

Se comprueba que la función sea correcta.

In [46]:
prueba_edu = ['bachelor\'s degree', 'secondary education']
prueba_in = ['business', 'employee']
prueba_age = ['20-29', '30-39']
prueba_total = [float('nan'), 102]
prueba = pd.DataFrame()
prueba['education'] = prueba_edu
prueba['income_type'] = prueba_in
prueba['age_group'] = prueba_age
prueba['total_income'] = prueba_total

print(prueba)
print()
prueba['total_income'] = prueba.apply(nan_median, axis=1)
print(prueba)

             education income_type age_group  total_income
0    bachelor's degree    business     20-29           NaN
1  secondary education    employee     30-39         102.0

             education income_type age_group  total_income
0    bachelor's degree    business     20-29     28250.284
1  secondary education    employee     30-39       102.000


Se aplica la función a la tabla original para restaurar los valores ausentes y se imprime la distribución de los valores ausentes por columna.

In [47]:
df['total_income'] = df.apply(nan_median, axis=1)
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           3
purpose                0
age_group              0
dtype: int64


Aún existen 3 filas con valores ausentes en `'total_income'`. Se imprimirán dichas filas para obtener información.

In [48]:
print(df[df['total_income'].isna()])

      children  days_employed  dob_years          education  education_id  \
1303         1            NaN         70  primary education             3   
5936         0            NaN         58  bachelor's degree             0   
8142         0            NaN         64  primary education             3   

          family_status  family_status_id gender    income_type  debt  \
1303  civil partnership                 1      F       employee     0   
5936            married                 0      M   entrepreneur     0   
8142  civil partnership                 1      F  civil servant     0   

      total_income         purpose age_group  
1303           NaN  for real state       70+  
5936           NaN  for real state     50-59  
8142           NaN   for a wedding     60-69  


Se localizarán las filas de la tabla sin valores ausentes que cumplen las características de la primera fila para comprobar si existen.

In [49]:
print(df_non_nan.loc[(df_non_nan['age_group'] == '70+') & (df_non_nan['education'] == 'primary education') & (df_non_nan['income_type'] == 'employee')])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, age_group]
Index: []


No existen clientes sin valores ausentes que cumplan estas tres características, por ello no se puede obtener la mediana. Como el ingreo mensual depende en mayor medida del puesto y la experiencia que del nivel educativo, se analizará la media y mediana de los ingresos menduales de los clientes que satisfacen tener más de 70 años y ser empleados.

In [50]:
df_70_employee = df_non_nan.loc[(df_non_nan['age_group'] == '70+') & (df_non_nan['income_type'] == 'employee')]
mean_df_70_employee = df_70_employee['total_income'].mean()
median_df_70_employee = df_70_employee['total_income'].median()
print('media:', mean_df_70_employee)
print('mediana:', median_df_70_employee)

media: 26672.38242857143
mediana: 24660.901


Nuevamente, estas cantidades difieren, es decir, existen valores atípicos por lo que se reemplazará con la mediana.

In [51]:
df.loc[1303, 'total_income'] = median_df_70_employee

En la tabla original sólo se tienen dos clientes con el valor de `'entrepreneur'`, esto quiere decir que uno de ellos tienen valores ausentes y el otro no. Por esta razón se reemplazará con los datos del otro cliente.

In [52]:
df_entrepreneur = df_non_nan.loc[df_non_nan['income_type'] == 'entrepreneur']
mean_entrepreneur = df_entrepreneur['total_income'].mean()
df.loc[5936, 'total_income'] = mean_entrepreneur

Para la tercera fila se procede como en el caso de la primera.

In [53]:
df_60_civil = df_non_nan.loc[(df_non_nan['age_group'] == '60-69') & (df_non_nan['income_type'] == 'civil servant')]
mean_df_60_civil = df_60_civil['total_income'].mean()
median_df_60_civil = df_60_civil['total_income'].median()
print('media:', mean_df_60_civil)
print('mediana:', median_df_60_civil)

media: 29305.166039215685
mediana: 23390.057


In [54]:
df.loc[8142, 'total_income'] = median_df_70_employee

Se comprueba que la columna `'total_income'`no contenga valores ausentes.

In [55]:
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_group              0
dtype: int64


[Volver a Contenidos](#back)

#### 2.3.2 Restaurar valores ausentes en `'days_employed'` <a id='data_preprocessing_missing_values_days'></a>

See restaurarán los valores ausentes de la columna `'days_employed'`.

La experiencia laboral de un cliente depende de tres facores: edad, sueldo y puesto. Se considerarán tablas dinámicas considerando estos tres factores y obteniendo las medias y medianas de los ingresos mensuales. Para ello se crearán grupos de ingreso mensual (como se hizo con la edad).

Primero se creará la función.

In [56]:
def income_group(income):
    if income < 10000:
        return '< 10K'
    elif 10000 <= income < 20000:
        return '10K-20K'
    elif 20000 <= income < 30000:
        return '20K-30K'
    elif 30000 <= income < 40000:
        return '30K-40K'
    elif 40000 <= income < 50000:
        return '40K-50K'
    else:
        return '50K+'

Se aplica la función a la columna `'total_income'` de la tabla original y se obtiene su información para comprobar que los cambios se hayan hecho correctamente.

In [57]:
df['income_group'] = df['total_income'].apply(income_group)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  age_group         21525 non-null  object 
 13  income_group      21525 non-null  object 
dtypes: float64(2), int64(5), object(7)
memory usage: 2.3+ MB


Se actualiza la tabla sin valores ausentes.

In [58]:
df_non_nan = df.dropna()

Se crea una tablas dinámicas con las medias y medianas de los días de experiencia laboral, considerando: edad, sueldo y puesto.

In [59]:
# Tabla dinámica de medias
data_pivot_mean_2 = df_non_nan.pivot_table(index=['age_group', 'income_group'], columns='income_type', values='days_employed', aggfunc='mean')
print(data_pivot_mean_2)

income_type                 business  civil servant     employee  \
age_group income_group                                             
20-29     10K-20K        1022.760561    1469.492662  1155.429146   
          20K-30K        1113.848812    1553.659614  1135.098094   
          30K-40K        1159.667909    1873.416503  1321.090301   
          40K-50K        1448.482955    1770.500433  1432.747555   
          50K+           1297.651942    1562.521355  1555.937608   
          < 10K           775.775780    1694.143804  1184.776317   
30-39     10K-20K        1790.047492    2695.299214  1883.721541   
          20K-30K        1835.205687    2918.917251  2044.918358   
          30K-40K        1892.646599    2609.679244  2101.929441   
          40K-50K        1991.686445    3276.796453  2096.383838   
          50K+           1798.916344    2692.706641  2173.003718   
          < 10K          2028.127688    2084.596160  2029.547161   
40-49     10K-20K        2439.558562    4260.595

In [60]:
# Tabla dinámica de medianas
data_pivot_median_2 = df_non_nan.pivot_table(index=['age_group', 'income_group'], columns='income_type', values='days_employed', aggfunc='median')
print(data_pivot_median_2)

income_type                 business  civil servant     employee  \
age_group income_group                                             
20-29     10K-20K         792.285881    1249.359709   963.687928   
          20K-30K         873.643137    1344.015982   974.402391   
          30K-40K         982.297458    1719.764110  1049.070884   
          40K-50K        1169.895269    1124.128998  1167.710588   
          50K+           1094.882749    1265.117765  1321.735258   
          < 10K           556.305202    1561.605375  1013.920085   
30-39     10K-20K        1376.274801    2522.413844  1462.129921   
          20K-30K        1520.197545    2679.721255  1571.872786   
          30K-40K        1629.705609    2666.912661  1546.037556   
          40K-50K        1506.456938    2999.903512  1746.500182   
          50K+           1551.358557    2377.533163  1763.577520   
          < 10K          1939.045288     910.610698  1407.902748   
40-49     10K-20K        1819.043373    3668.973

También en este caso se utilizará las medianas debido a la diferencia que tiene con las medias (existen valores atípicos).

In [61]:
# Función para reemplazar valores
def nan_median_days(row):
    days = row['days_employed']
    age = row['age_group']
    income = row['income_type']
    in_group = row['income_group']
    if pd.isna(days):
        days = data_pivot_median_2.loc[(age, in_group), income]
        return days
    else:
        return days

Se comprueba que la función es correcta.

In [62]:
# Comprobación de que la función es correcta
proof_days = [float('nan'), 120]
proof_in = ['business', 'employee']
proof_age = ['20-29', '30-39']
proof_group = ['10K-20K', '<10K']
proof = pd.DataFrame()
proof['days_employed'] = proof_days
proof['income_type'] = proof_in
proof['age_group'] = proof_age
proof['income_group'] = proof_group

print(proof)
print()
proof['days_employed'] = proof.apply(nan_median_days, axis=1)
print(proof)

   days_employed income_type age_group income_group
0            NaN    business     20-29      10K-20K
1          120.0    employee     30-39         <10K

   days_employed income_type age_group income_group
0     792.285881    business     20-29      10K-20K
1     120.000000    employee     30-39         <10K


Se aplica la función a la tabla y se verifican los valores ausentes por columnas.

In [63]:
df['days_employed'] = df.apply(nan_median_days, axis=1)
print(df.isna().sum())

children            0
days_employed       1
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
income_group        0
dtype: int64


Aún existe una fila con valor ausente. Se imprime dicha fila para observación.

In [64]:
print(df[df['days_employed'].isna()])

      children  days_employed  dob_years          education  education_id  \
5936         0            NaN         58  bachelor's degree             0   

     family_status  family_status_id gender   income_type  debt  total_income  \
5936       married                 0      M  entrepreneur     0     79866.103   

             purpose age_group income_group  
5936  for real state     50-59         50K+  


Nuevamente es la fila con el cliente cuyo valor en `'income_type'` es entrepreneur. Se reemplazará con los datos del otro cliente y se verifican los valores ausentes por columna.

In [65]:
mean_days_entrepreneur = df_entrepreneur['days_employed'].mean()
df.loc[5936, 'days_employed'] = mean_days_entrepreneur
print(df.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
income_group        0
dtype: int64


Ya no existen valores ausentes.

[Volver a Contenidos](#back)

### 2. 4. Conclusiones <a id='data_preprocessing_conclusions'></a>

Se encontraron duplicados implicítos en la mayoría de columnas de tipo object y se corrigieron aquellos que diferían por letras mayúsculas. También se agruparon aquellos que contenían sinónimos o frases que trataban el mismo propósito.

Se trataron los datos problemáticos de las columnas con valores de tipo int o float. Además se decidió mantener las filas duplicadas al carecer de un identificador de cliente y por tanto desconocer si realmente se trata del mismo cliente o no.

Finalmente se restauraron los valores ausentes con las medianas con factores correspondientes.

[Volver a Contenidos](#back)

## Etapa 3. Comprobación de hipótesis <a id='hypotheses'></a>

En esta etapa se comprobará la veracidad de las hipótesis planteadas en la introducción, las cuales son:
1. El incumplimiento de pago de un préstamo de los clientes difiere según el estado civil, cantidad de hijos, ingreso mensual o propósito del pago. 
2. Los clientes con una cantidad menor de hijos forman el mayor porcentaje de pago a tiempo.
3. Los clientes casados forman el mayor porcentaje de pago a tiempo.
4. Los clientes con mayores ingresos forman el mayor porcentaje de pago a tiempo.
5. Los propósitos del préstamo no interfieren con el pago a tiempo.

Ya se han clasificado los datos por rangos de edad y rangos de ingreso mensual. La última clasificación útil será por cantidad de hijos del cliente. Se procede con dicha clasificación.

In [66]:
# Función para clasificar cantidad de hijos
def children_group(number):
    if number == 0:
        return 'sin hijos'
    elif 1 <= number < 3:
        return '1 o 2 hijos'
    elif 3<= number < 5:
        return '3 o 4 hijos'
    else:
        return '5 o más hijos'

Se agrega la columna `'children_group'` a la tabla utilizando la función recién creada. Se obtiene la información de la tabla para comprobar que los cambios se hayan realizado correctamente.

In [67]:
df['children_group'] = df['children'].apply(children_group)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  age_group         21525 non-null  object 
 13  income_group      21525 non-null  object 
 14  children_group    21525 non-null  object 
dtypes: float64(2), int64(5), object(8)
memory usage: 2.5+ MB


También se agregará la columna `'late_payment'` que describirá si el cliente ha tenido alguna deuda o no.

In [68]:
def deuda(number):
    if number == 0:
        return 'No'
    else:
        return 'Si'
df['late_payment'] = df['debt'].apply(deuda)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  age_group         21525 non-null  object 
 13  income_group      21525 non-null  object 
 14  children_group    21525 non-null  object 
 15  late_payment      21525 non-null  object 
dtypes: float64(2), int64(5), object(9)
memor

[Volver a Contenidos](#back)

### 3.1. Hipótesis: cantidad de hijos <a id='hypotheses_children'></a>

Para comenzar, se obtendrán las cantidades de clientes que han tenido alguna deuda y los que no.

In [69]:
df_debt = df['late_payment'].value_counts()
print(df_debt)
print()
debt = df_debt[1]
conversion_debt = debt / len(df)
print(f'Hay {debt} clientes que han tenido alguna deuda, esto es el {conversion_debt:.0%}')

No    19784
Si     1741
Name: late_payment, dtype: int64

Hay 1741 clientes que han tenido alguna deuda, esto es el 8%


Tenemos que el 8% de los clientes han tenido alguna deuda en el pago de un prestámo, a esto se le llamará tendencia general.

A continuación se trabajará con una tabla dinámica para conocer como influye la cantidad de hijos con el pago del préstamo.

In [70]:
df_hip_children = df.pivot_table(index='children_group', columns='late_payment', values='debt', aggfunc='count')

df_hip_children['late_payment'] = (df_hip_children['Si'] / (df_hip_children['No'] + df_hip_children['Si'])) * 100
df_hip_children['percent_on_time'] = (df_hip_children['No'] / df_hip_children['No'].sum()) * 100
print(df_hip_children)

late_payment       No    Si  late_payment  percent_on_time
children_group                                            
1 o 2 hijos      6235   638      9.282700        31.515366
3 o 4 hijos       340    31      8.355795         1.718560
5 o más hijos      77     8      9.411765         0.389203
sin hijos       13132  1064      7.495069        66.376870


En la tabla se han agregado dos columnas. En `'late_payment'` se muestran los porcentajes de los clientes con alguna deuda por categoría (cantidad de hijos). En `'percent_on_time'` se muestran los porcentajes de clientes sin deuda de la categoría correspondiente con respecto a la cantidad total de clientes sin deuda.

Como se puede apreciar en `'late_payment'`los porcentajes no difieren en más de 2% de la tendencia general. Por esta razón concluimos que la cantidad de hijos no es un factor que influya en el incumplimiento del pago de un prestámo.

Por otro lado, como se aprecia en `'percent_on_time'`, el mayor porcentaje (66%) de clientes sin deuda está conformado por clientes sin hijos, seguidos por aquellos que tienen 1 o 2 hijos (31%).

[Volver a Contenidos](#back)

### 3.2. Hipótesis: estado civil <a id='hypotheses_family_status'></a>

Se obtendrá una tabla dinámica, similar a la anterior para conocer como influye el estado civil con el pago de un prestámo.

In [71]:
df_hip_family = df.pivot_table(index='family_status', columns='late_payment', values='debt', aggfunc='count')

df_hip_family['late_payment'] = (df_hip_family['Si'] / (df_hip_family['No'] + df_hip_family['Si'])) * 100
df_hip_family['percent_on_time'] = (df_hip_family['No'] / df_hip_family['No'].sum()) * 100
print(df_hip_family)

late_payment          No   Si  late_payment  percent_on_time
family_status                                               
civil partnership   3789  388      9.288963        19.151840
divorced            1110   85      7.112971         5.610594
married            11449  931      7.520194        57.869996
unmarried           2539  274      9.740491        12.833603
widow / widower      897   63      6.562500         4.533967


Como se puede apreciar en `'late_payment'` los porcentajes no difieren en más de 2% de la tendencia general. Por esta razón concluimos que el estado civil no es un factor que influya en el incumplimiento del pago de un prestámo.

Por otro lado, como se aprecia en `'percent_on_time'`, el mayor porcentaje (57%) de clientes sin deuda está conformado por clientes casados, seguidos por aquellos que tienen una unión civil (19%).

[Volver a Contenidos](#back)

### 3.3. Hipótesis: ingreso mensual <a id='hypotheses_total_income'></a>

Se obtendrá una tabla dinámica, similar a la anterior para conocer como influye el ingreso mensual con el pago de un prestámo.

In [72]:
df_hip_income = df.pivot_table(index='income_group', columns='late_payment', values='debt', aggfunc='count')

df_hip_income['late_payment'] = (df_hip_income['Si'] / (df_hip_income['No'] + df_hip_income['Si'])) * 100
df_hip_income['percent_on_time'] = (df_hip_income['No'] / df_hip_income['No'].sum()) * 100
print(df_hip_income)

late_payment    No   Si  late_payment  percent_on_time
income_group                                          
10K-20K       6211  581      8.554181        31.394056
20K-30K       7062  659      8.535164        35.695512
30K-40K       3024  249      7.607699        15.285079
40K-50K       1390  102      6.836461         7.025879
50K+          1229   92      6.964421         6.212091
< 10K          868   58      6.263499         4.387384


Como se puede apreciar en `'late_payment'` los porcentajes no difieren en más de 2% de la tendencia general. Por esta razón concluimos que el ingreso mensual no es un factor que influya en el incumplimiento del pago de un prestámo.

Por otro lado, como se aprecia en `'percent_on_time'`, el mayor porcentaje (35%) de clientes sin deuda está conformado por clientes con un ingreso mensual entre 20 y 30 mil, seguidos por aquellos con un ingreso mensual entre 10 y 20 mil (19%).

[Volver a Contenidos](#back)

### 3.4. Hipótesis: propósito del prestámo <a id='hypotheses_purpose'></a>

Se obtendrá una tabla dinámica, similar a la anterior para conocer como influye el propósito del prestámo con el pago del mismo.

In [73]:
df_hip_purpose = df.pivot_table(index='purpose', columns='late_payment', values='debt', aggfunc='count')

df_hip_purpose['late_payment'] = (df_hip_purpose['Si'] / (df_hip_purpose['No'] + df_hip_purpose['Si'])) * 100
df_hip_purpose['percent_on_time'] = (df_hip_purpose['No'] / df_hip_purpose['No'].sum()) * 100
print(df_hip_purpose)

late_payment       No   Si  late_payment  percent_on_time
purpose                                                  
for a wedding    2162  186      7.921635        10.928023
for education    3652  370      9.199403        18.459361
for real state  10058  782      7.214022        50.839062
to buy a car     3912  403      9.339513        19.773554


Como se puede apreciar en `'late_payment'` los porcentajes no difieren en más de 2% de la tendencia general. Por esta razón concluimos que el propósito del prestámo no es un factor que influya en el incumplimiento del pago del mismo.

Por otro lado, como se aprecia en `'percent_on_time'`, el mayor porcentaje (50%) de clientes sin deuda está conformado por clientes con el propósito de comprar, rentar o renovar una propiedad, seguidos por aquellos con el propósito de comprar un auto (19%).

[Volver a Contenidos](#back)

### 3.5. Conclusiones <a id='hypotheses_conclusions'></a>

La primera hipótesis:

**1. El incumplimiento de pago de un préstamo de los clientes difiere según el estado civil, cantidad de hijos, ingreso mensual o propósito del pago.**

Es cierta aunque como se ha demostrado, estos factores no difieren por mucho de la tendencia general.

La segunda hipótesis:

**2. Los clientes con una cantidad menor de hijos forman el mayor porcentaje de pago a tiempo.**

Es cierta como se ha demostrado.

La tercera hipótesis:

**3. Los clientes casados forman el mayor porcentaje de pago a tiempo.**

También es cierta.

La cuarta hipótesis:

**4. Los clientes con mayores ingresos forman el mayor porcentaje de pago a tiempo.**

Es falsa, el mayor porcentaje de clientes sin deuda está formado por clientes con un ingreso mensual de entre 10 y 30 mil.

Finalmente, la última hipótesis:

**5. Los propósitos del préstamo no interfieren con el pago a tiempo.**

También es falsa, el mayor porcentaje de los clientes sin deuda pertenece a aquellos con el propósito de comprar, rentar o renovar una propiedad. Sin embargo, los demás propósitos tienen un porcentaje similar. Se podría concluir que sin contar los propósitos referentes a una casa, el propósito no influye con el pago del prestámo.

[Volver a Contenidos](#back)

## Conclusión general <a id='conclusions'></a>

Se probaron tres de las cinco hipótesis planteadas en un principio. Sin embargo, se debe considerar que la fuente original contenía datos ausentes que fueron restaurados con las medianas correspondientes. Además, los datos no contienen un identificador de cliente, por lo que también se tienen filas duplicadas e imposibles de determinar se pertenecen a un mismo cliente o no. Considerando esto las conclusiones sobre las hipótesis podrían diferir.

Basados en este proyecto, se puede concluir que los clientes con mejor puntuación de crédito son aquellos clientes casados (o en unión civil), con a lo más 2 hijos, con un ingreso mensual de entre 10 y 30 mil y con el propósito de comoprar, rentar o renovar una propiedad.